In [1]:
using Plots

In [ ]:


function grid_circle(; Δr::Float64, Δθ::Float64, r::Float64)
    rs = 0.0:Δr:r
    θs = 0.0:Δθ:2π
    nr, nθ = length(rs), length(θs)
    rr = rs' .* ones(nr)
    θθ = θs' .* ones(nθ)
    return (nr, nθ, rr, θθ)
end;



In [2]:




function wave2D_FFD_circle(; Δr::Float64, 
                           Δθ::Float64,
                           Δt::Float64, 
                           c::Float64, 
                           N::Int64,
                           ICs::Dict, 
                           BCs::Dict, 
                           grid_params::Tuple)

    (nr, nθ, rr, θθ) = grid_params
    i = collect(2:nr-1)
    j = collect(2:nθ-1)
    
    u_Ay, u_By = BCs["u(A, y, t)"], BCs["u(B, y, t)"]
    u_xC, u_xD = BCs["u(x, C, t)"], BCs["u(x, D, t)"]
    I, V = ICs["I(x, y)"], ICs["V(x, y)"]

    rr_ = c * Δt / Δr
    rθ = c * Δt / Δθ

#     if rx^2 + ry^2 > 1 # checking if method is stable
#         (@warn "Stability condition is not met! (rx^2 + ry^2 > 1)")
#     end

    u = zeros(nr, nθ, N)
    u[:,  :, 1] = I.(xx, yy)                                                        #   <- first IC (t = 0)
    u[i,  j, 2] = (u[i, j, 1] .+ Δt .* V.(xx, yy)[i, j]                             # }
                .+ rx^2/2 * (u[i.+1, j, 1] .- 2 .* u[i, j, 1] .+ u[i.-1, j, 1])     # } <- second IC (t = 1)
                .+ ry^2/2 * (u[i, j.+1, 1] .- 2 .* u[i, j, 1] .+ u[i, j.-1, 1]))    # }

    u[1,  :, 1:2] .= u_Ay  # }
    u[:,  1, 1:2] .= u_xC  # } <- putting BCs
    u[ny, :, 1:2] .= u_By  # }
    u[:, nx, 1:2] .= u_xD  # }

    for t in 3:N # Solving for t = 2, 3, ..., T
        u[i, j, t] = (2 .* u[i, j, t.-1] .- u[i, j, t.-2]
                    .+ rx^2 * (u[i.+1, j, t.-1] .- 2 .* u[i, j, t.-1] .+ u[i.-1, j, t.-1]) 
                    .+ ry^2 * (u[i, j.+1, t.-1] .- 2 .* u[i, j, t.-1] .+ u[i, j.-1, t.-1]))              
    end
    return u
end;

